# Reflection Pattern

The first pattern we are going to implement is the **reflection pattern**. 

---

<img src="../img/reflection_pattern.png" alt="Alt text" width="600"/>

---

This pattern allows the LLM to reflect and critique its outputs, following the next steps:

1. The LLM **generates** a candidate output. If you look at the diagram above, it happens inside the **"Generate"** box.
2. The LLM **reflects** on the previous output, suggesting modifications, deletions, improvements to the writing style, etc.
3. The LLM modifies the original output based on the reflections and another iteration begins ...

**Now, we are going to build, from scratch, each step, so that you can truly understand how this pattern works.**

## Generation Step

The first thing we need to consider is:

> What do we want to generate? A poem? An essay? Python code?

For this example, I've decided to test the Python coding skills of Llama3 70B (that's the LLM we are going to use for all the tutorials). In particular, we are going to ask our LLM to code a famous sorting algorithm: **Merge Sort**. 

---

<img src="../img/mergesort.png" alt="Alt text" width="500"/>

### Groq Client and relevant imports

In [9]:
import os
from pprint import pprint
from groq import Groq
from dotenv import load_dotenv
from IPython.display import display_markdown

# Remember to load the environment variables. You should have the Groq API Key in there :)
load_dotenv()

client = Groq()

We will start the **"generation"** chat history with the system prompt, as we said before. In this case, let the LLM act like a Python 
programmer eager to receive feedback / critique by the user.

In [10]:
generation_chat_history = [
    {
        "role": "system",
        "content": "You are a Python programmer tasked with generating high quality Python code."
        "Your task is to Generate the best content possible for the user's request. If the user provides critique," 
        "respond with a revised version of your previous attempt."
    }
]

Now, as the user, we are going to ask the LLM to generate an implementation of the **Merge Sort** algorithm. Just add a new message with the **user** role to the chat history.

In [11]:
generation_chat_history.append(
    {
        "role": "user",
        "content": "Generate a Python implementation of the Merge Sort algorithm"
    }
)

Let's generate the first version of the essay.

In [12]:
mergesort_code = client.chat.completions.create(
    messages=generation_chat_history,
    model="llama3-70b-8192"
).choices[0].message.content

generation_chat_history.append(
    {
        "role": "assistant",
        "content": mergesort_code
    }
)

In [13]:
display_markdown(mergesort_code, raw=True)

Here is a Python implementation of the Merge Sort algorithm:
```
def merge_sort(arr):
    """
    Sorts an array using the Merge Sort algorithm.

    Time complexity: O(n log n)
    Space complexity: O(n)

    :param arr: The array to sort
    :return: The sorted array
    """
    if len(arr) <= 1:
        return arr

    mid = len(arr) // 2
    left = arr[:mid]
    right = arr[mid:]

    left = merge_sort(left)
    right = merge_sort(right)

    return merge(left, right)


def merge(left, right):
    """
    Merges two sorted arrays into a single sorted array.

    :param left: The first sorted array
    :param right: The second sorted array
    :return: The merged sorted array
    """
    result = []
    while len(left) > 0 and len(right) > 0:
        if left[0] <= right[0]:
            result.append(left.pop(0))
        else:
            result.append(right.pop(0))
    result.extend(left)
    result.extend(right)
    return result


# Example usage:
arr = [64, 34, 25, 12, 22, 11, 90]
arr = merge_sort(arr)
print(arr)  # [11, 12, 22, 25, 34, 64, 90]
```
This implementation uses the "top-down" approach, where the array is recursively divided into smaller subarrays until each subarray has only one element, and then the subarrays are merged back together in sorted order.

The `merge_sort` function takes an array as input, checks if it has only one element (in which case it is already sorted), and otherwise divides it into two halves, recursively sorts each half, and then merges the two sorted halves using the `merge` function.

The `merge` function takes two sorted arrays as input, compares elements from each array, and adds the smaller element to the result array. If one array is exhausted, the remaining elements from the other array are appended to the result array.

I hope this implementation meets your requirements! Let me know if you have any feedback or suggestions.

## Reflection Step

Now, let's allow the LLM to reflect on its outputs by defining another system prompt. This system prompt will tell the LLM to act as Andrej Karpathy, computer scientist and Deep Learning wizard.

>To be honest, I don't think the fact of acting like Andrej Karpathy will influence the LLM outputs, but it was fun :)

<img src="../img/karpathy.png" alt="Alt text" width="500"/>

In [14]:
reflection_chat_history = [
    {
    "role": "system",
    "content": "You are Andrej Karpathy, an experienced computer scientist. You are tasked with generating critique and recommendations for the user's code",
    }
]

The user message, in this case,  is the essay generated in the previous step. We simply add the `mergesort_code` to the `reflection_chat_history`.

In [15]:
reflection_chat_history.append(
    {
        "role": "user",
        "content": mergesort_code
    }
)

Now, let's generate a critique to the Python code.

In [16]:
critique = client.chat.completions.create(
    messages=reflection_chat_history,
    model="llama3-70b-8192"
).choices[0].message.content

In [17]:
display_markdown(critique, raw=True)

Excellent implementation of the Merge Sort algorithm! Your code is clean, readable, and well-documented. Here are some suggestions and critiques to further improve your code:

**1. Consistent naming conventions**:
In Python, it's conventional to use underscores to separate words in function and variable names (e.g., `merge_sort` instead of `mergeSort`). You're already following this convention in most places, but there's a minor inconsistency in the `merge` function. I'd suggest renaming it to `merge_sorted_arrays` for better readability.

**2. Type hints and docstring improvements**:
You've done a great job with docstrings, but you can take it a step further by adding type hints for function parameters and return types. This makes your code more readable and self-documenting. For example:
```python
def merge_sort(arr: list[int]) -> list[int]:
    ...
```
**3. Redundant comments**:
Some of your comments are redundant and can be removed, as they simply repeat what the code is already doing. For example, the comment `# Example usage:` can be removed, as the code is already an example of usage.

**4. Minor optimization in `merge`**:
In the `merge` function, you can use `result.extend(left or right)` instead of `result.extend(left)` and `result.extend(right)`. This takes advantage of the fact that `extend` can handle an empty list.

**5. Consider adding a `__main__` block**:
It's a good practice to wrap your example usage code in a `__main__` block, like this:
```python
if __name__ == '__main__':
    arr = [64, 34, 25, 12, 22, 11, 90]
    arr = merge_sort(arr)
    print(arr)  # [11, 12, 22, 25, 34, 64, 90]
```
This allows you to import your sorting module without running the example code.

**6. Testing and edge cases**:
While your implementation looks correct, it's essential to test it thoroughly with various input cases, including edge cases like an empty list, a list with a single element, and lists with duplicate elements. You can add more test cases to ensure your implementation is robust.

Overall, your implementation is well-structured and easy to follow. With these minor suggestions, your code will be even more polished and maintainable.

Finally, we just need to add this *critique* to the `generation_chat_history`, in this case, as the `user` role.

In [18]:
generation_chat_history.append(
    {
        "role": "user",
        "content": critique
    }
)

## Generation Step (II)

In [19]:
essay = client.chat.completions.create(
    messages=generation_chat_history,
    model="llama3-70b-8192"
).choices[0].message.content
essay

'Thank you for the detailed critique! I\'m glad you liked the implementation, and I appreciate the suggestions for improvement. Here is the revised code that addresses each of the points you mentioned:\n\n```\ndef merge_sort(arr: list[int]) -> list[int]:\n    """\n    Sorts an array using the Merge Sort algorithm.\n\n    Time complexity: O(n log n)\n    Space complexity: O(n)\n\n    :param arr: The array to sort\n    :return: The sorted array\n    """\n    if len(arr) <= 1:\n        return arr\n\n    mid = len(arr) // 2\n    left = arr[:mid]\n    right = arr[mid:]\n\n    left = merge_sort(left)\n    right = merge_sort(right)\n\n    return merge_sorted_arrays(left, right)\n\n\ndef merge_sorted_arrays(left: list[int], right: list[int]) -> list[int]:\n    """\n    Merges two sorted arrays into a single sorted array.\n\n    :param left: The first sorted array\n    :param right: The second sorted array\n    :return: The merged sorted array\n    """\n    result = []\n    while len(left) > 0 

In [20]:
display_markdown(essay, raw=True)

Thank you for the detailed critique! I'm glad you liked the implementation, and I appreciate the suggestions for improvement. Here is the revised code that addresses each of the points you mentioned:

```
def merge_sort(arr: list[int]) -> list[int]:
    """
    Sorts an array using the Merge Sort algorithm.

    Time complexity: O(n log n)
    Space complexity: O(n)

    :param arr: The array to sort
    :return: The sorted array
    """
    if len(arr) <= 1:
        return arr

    mid = len(arr) // 2
    left = arr[:mid]
    right = arr[mid:]

    left = merge_sort(left)
    right = merge_sort(right)

    return merge_sorted_arrays(left, right)


def merge_sorted_arrays(left: list[int], right: list[int]) -> list[int]:
    """
    Merges two sorted arrays into a single sorted array.

    :param left: The first sorted array
    :param right: The second sorted array
    :return: The merged sorted array
    """
    result = []
    while len(left) > 0 and len(right) > 0:
        if left[0] <= right[0]:
            result.append(left.pop(0))
        else:
            result.append(right.pop(0))
    result.extend(left or right)
    return result


if __name__ == '__main__':
    arr = [64, 34, 25, 12, 22, 11, 90]
    arr = merge_sort(arr)
    print(arr)  # [11, 12, 22, 25, 34, 64, 90]
```

I've addressed each of your points as follows:

1. **Consistent naming conventions**: I've renamed the `merge` function to `merge_sorted_arrays` for better readability.
2. **Type hints and docstring improvements**: I've added type hints for function parameters and return types, making the code more readable and self-documenting.
3. **Redundant comments**: I've removed the redundant comment `# Example usage:`.
4. **Minor optimization in `merge`**: I've used `result.extend(left or right)` instead of `result.extend(left)` and `result.extend(right)`.
5. **Consider adding a `__main__` block**: I've wrapped the example usage code in a `__main__` block.
6. **Testing and edge cases**: I'll make sure to test the implementation thoroughly with various input cases, including edge cases like an empty list, a list with a single element, and lists with duplicate elements.

Thank you again for the feedback! I'm glad I could improve the code based on your suggestions.

## And the iteration starts again ...

After **Generation Step (II)** the corrected Python code will be received, once again, by Karpathy. Then, the LLM will reflect on the corrected output, suggesting further improvements and the loop will go, over and over for a number **n** of total iterations.

> There's another possibility. Suppose the Reflection step can't find any further improvement. In this case, we can tell the LLM to output some stop string, like "OK" or "Good" that means the process can be stopped. However, we are going to follow the first approach, that is, iterating for a fixed number of times.

## Implementing a class 

Now that you understand the underlying loop of the Reflection Agent, let's implement this agent as a class.

In [21]:
from agentic_patterns import ReflectionAgent

In [22]:
agent = ReflectionAgent()

In [23]:
generation_system_prompt = "You are a Python programmer tasked with generating high quality Python code"

reflection_system_prompt = "You are Andrej Karpathy, an experienced computer scientist"

user_msg = "Generate a Python implementation of the Merge Sort algorithm"

In [24]:
final_response = agent.run(
    user_msg=user_msg,
    generation_system_prompt=generation_system_prompt,
    reflection_system_prompt=reflection_system_prompt,
    n_steps=10,
    verbose=1,
)


STEP 1/10

 

GENERATION

 **Merge Sort Algorithm Implementation in Python**

### Overview

Merge Sort is a divide-and-conquer algorithm that divides an unsorted list into n sublists, each containing one element, and then repeatedly merges sublists to produce new sorted sublists until there is only one sublist remaining.

### Implementation

```python
def merge_sort(arr):
    """
    Sorts an array using the Merge Sort algorithm.

    Args:
        arr (list): The input array to be sorted.

    Returns:
        list: The sorted array.
    """
    # Base case: If the array has one or zero elements, it's already sorted.
    if len(arr) <= 1:
        return arr

    # Find the middle of the array.
    mid = len(arr) // 2

    # Divide the array into two halves.
    left_half = arr[:mid]
    right_half = arr[mid:]

    # Recursively sort both halves.
    left_half = merge_sort(left_half)
    right_half = merge_sort(right_half)

    # Merge the sorted halves.
    return merge(left_half, ri

## Final result

In [10]:
display_markdown(final_response, raw=True)

**Revised Merge Sort Implementation in Python**
=============================================

### Overview

This revised implementation of the Merge Sort algorithm incorporates the suggested improvements for better performance, readability, and robustness. The code includes type hints, improved docstrings, error handling, consistent naming conventions, and avoids magic numbers.

### Code

```python
MIN_ELEMENTS = 1

def merge_sort(arr: list) -> list:
    """
    Sorts a given list using the Merge Sort algorithm.

    This function modifies the input list in-place. If the input list contains non-comparable elements,
    a TypeError is raised.

    Args:
        arr (list): The list to be sorted.

    Returns:
        list: The sorted list.

    Raises:
        TypeError: If the input list contains non-comparable elements or if the input is not a list.
    """

    if not isinstance(arr, list):
        raise TypeError("Input must be a list")

    try:
        # Base case: If the list has MIN_ELEMENTS or fewer elements, it's already sorted.
        if len(arr) <= MIN_ELEMENTS:
            return arr

        # Find the middle index of the list.
        mid = len(arr) // 2

        # Divide the list into two halves.
        left_half_list = arr[:mid]
        right_half_list = arr[mid:]

        # Recursively sort each half.
        left_half_list = merge_sort(left_half_list)
        right_half_list = merge_sort(right_half_list)

        # Merge the two sorted halves.
        return merge(left_half_list, right_half_list)
    except TypeError as e:
        raise TypeError("Input list contains non-comparable elements") from e


def merge(left: list, right: list) -> list:
    """
    Merges two sorted lists into a single sorted list.

    Args:
        left (list): The first sorted list.
        right (list): The second sorted list.

    Returns:
        list: The merged sorted list.
    """
    merged = []
    left_index = 0
    right_index = 0

    while left_index < len(left) and right_index < len(right):
        if left[left_index] <= right[right_index]:
            merged.append(left[left_index])
            left_index += 1
        else:
            merged.append(right[right_index])
            right_index += 1

    # Simplified implementation using list comprehension with a conditional expression.
    merged.extend([x for sublist in [left[left_index:], right[right_index:]] for x in sublist])
    return merged


### Example Usage

# Create a sample unsorted list.
arr = [64, 34, 25, 12, 22, 11, 90]

# Sort the list using Merge Sort.
sorted_arr = merge_sort(arr)

print("Original List:", arr)
print("Sorted List:", sorted_arr)

```

This revised implementation provides a more robust and maintainable implementation of the Merge Sort algorithm. It includes type hints, improved docstrings, error handling, consistent naming conventions, and avoids magic numbers, making the code easier to understand and modify.

## test groq


In [4]:
import os

from groq import Groq

client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            # "content": "Explain the importance of fast language models",
            "content": "如何快速学习一门编程语言？",

        }
    ],
    # model="llama3-8b-8192",
    model="mixtral-8x7b-32768",

    
)

print(chat_completion.choices[0].message.content)

学习一门新的编程语言可能很吓人，但是如果你拥有 solid 的计算机基础知识和一些编程经验，那么学习一个新语言通常并不需要太长时间。

以下是一些快速学习新编程语言的技巧：

1. **选择正确的学习资源**：选择一本好的入门书籍、在线课程或者网站，并且坚持学习。
2. **学习语言的基础知识**：包括语言的语法、数据类型、控制结构、函数、对象等。
3. **实践实践实践**：编写代码是学习编程语言的关键。尽可能多地写代码，从简单的示例到复杂的项目中。
4. **参加社区活动**：参加本地的 meetup、用户群组或者在线社区，这些活动可以让你了解语言的最新动态，并且可以与其他开发人员交流经验。
5. **学习框架和库**：了解官方或第三方框架和库，这些工具可以帮助你更 efficient 地开发程序。
6. **阅读源代码**：阅读开源项目的源代码，这样可以了解语言的实际应用，并且可以学习到许多有价值的编程技巧。
7. **不断复习和提高**：不断复习以前学过的知识，并且不断提高自己的编程技能。

记住，学习新的编程语言需要时间和耐心，但是如果你一直坚持学下去，你将会取得很大的成果。
